# B2 again

In [16]:
# Sabaody
# Copyright 2018 Shaik Asifullah and J Kyle Medley

import arrow
from influxdb import InfluxDBClient

from numpy import array
import math

from pprint import pprint
from ast import literal_eval

client = InfluxDBClient('luna')
tstart = None
results = client.query('SELECT island_id,best_f,best_x,round FROM champion', database='com.how2cell.sabaody.biopredyn.b3-driver.5539fd10-447c-4feb-aed7-7702705c7dfe')
# pprint(results)
for result in results:
    for point in result:
        t = arrow.get(point['time'])
        if tstart == None or t < tstart:
            tstart = t
timepoints_by_island = {}
min_f = None
best_x = None
island_ids = set()
for result in results:
    for point in result:
        t = arrow.get(point['time'])-tstart
        round = int(point['round'])
        island_id = point['island_id']
        if not island_id in island_ids:
            island_ids.add(island_id)
        best_f = literal_eval(point['best_f'])[0]
#         best_f = math.log10(literal_eval(point['best_f'])[0])
#         best_f = 1./literal_eval(point['best_f'])[0]
        if min_f is None or best_f < min_f:
            min_f = best_f
            best_x = literal_eval(point['best_x'])
        x = t.seconds/3600.
#         x = round
        timepoints_by_island.setdefault(island_id, []).append((x,best_f))
print(island_ids)
print(len(island_ids))

import tellurium as te
for island_id,series in timepoints_by_island.items():
    trace = array(sorted(series, key=lambda t: t[0]))
    te.plot(trace[:,0], trace[:,1], tag='first', show=False)

print('first min obj',min_f)
print(best_x)

In [17]:
# client = InfluxDBClient('luna')
# tstart = None
# results = client.query('SELECT island_id,best_f,best_x,round FROM champion', database='com.how2cell.sabaody.biopredyn.b2-driver.82313aec-cfa3-43ad-b856-5ac309747135')
# # pprint(results)
# for result in results:
#     for point in result:
#         t = arrow.get(point['time'])
#         if tstart == None or t < tstart:
#             tstart = t
# timepoints_by_island = {}
# min_f = None
# best_x = None
# island_ids = set()
# for result in results:
#     for point in result:
#         t = arrow.get(point['time'])-tstart
#         round = int(point['round'])
#         island_id = point['island_id']
#         if not island_id in island_ids:
#             island_ids.add(island_id)
#         best_f = literal_eval(point['best_f'])[0]
# #         best_f = math.log10(literal_eval(point['best_f'])[0])
# #         best_f = 1./literal_eval(point['best_f'])[0]
#         if min_f is None or best_f < min_f:
#             min_f = best_f
#             best_x = literal_eval(point['best_x'])
#         x = t.seconds/3600.
# #         x = round
#         timepoints_by_island.setdefault(island_id, []).append((x,best_f))
#         # print(t.seconds,island_id,best_f)
# print(island_ids)
# print(len(island_ids))
        
# import tellurium as te
# for island_id,series in timepoints_by_island.items():
#     trace = array(sorted(series, key=lambda t: t[0]))
#     te.plot(trace[:,0], trace[:,1], tag='second', show=False)

# print('second min obj',min_f)

In [18]:
te.show()